In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.special import expit
import statsmodels.api as sm

# Part A) Parameter Estimation

In [2]:
pratt = pd.read_csv("Pratt/Pratt.csv")
y = pratt.choose.to_numpy()

x_cols = pratt.columns[~pratt.columns.isin(["choose"])]
X = pratt[x_cols].to_numpy()
X = np.c_[np.ones(X.shape[0]), X] # add intercept

print(y.shape)
print(X.shape)

(842,)
(842, 5)


In [4]:
def logit_score_fn(beta: np.ndarray, y: np.ndarray, X: np.ndarray): 
    """The derivative of the log-likelihood function for a binary 
    logit model. 

    $$\\sum_{i=1}^n (y_i - 1) + 1 / (1 + \\exp(-beta^T * x_i))) x_i $$

    Parameters 
    ----------
    beta: np.ndarray (k X 1)
        parameter array to optimize over
 
    y: np.ndarray (n X 1)  
        array of outcomes 

    X: np.ndarray (n X k)
        array of explanatory variables
    """
    # temporary varialbe to hold the value of -B^T * x
    nu = X @ beta 
    p = 1 / (1 + np.exp(-nu))

    return X.T @ (y - p) 


beta_0 = np.repeat(0, X.shape[1])
logit_score_fn(beta_0, y, X)

array([ 286.  ,  545.5 , 4574.75, 5652.  , -489.5 ])

In [6]:
sol = opt.root(lambda beta: logit_score_fn(beta, y, X), beta_0)
beta = sol.x

colnames = ["Intercept", *x_cols]
for col, est in zip(colnames, beta): 
    print(f"{col}: coef = {est}")

Intercept: coef = -1.2217718822913008
cars: coef = 2.308279763065528
dovtt: coef = 0.062225970359342055
divtt: coef = 0.009247957043049721
dcost: coef = 0.01694424853458004


# Part B) SE Estimation

In [8]:
def logit_score_by_obs(beta, y, X):
    """Calculates the logit score for each observation individually"""
    p = expit(X @ beta)
    s = (y - p)[:, None] * X
    return s


def hessian_logit(beta, X): 
    """Negative empirical hessian of the binary logit model."""
    p = expit(X @ beta)
    W = np.diag(p * (1 - p))
    A_hat = X.T @ W @ X

    return A_hat 


s = logit_score_by_obs(beta, y, X)

B_hat = (s.T @ s)
A_hat = hessian_logit(beta, X)

In [9]:
var_ests = {
    "Outer Product Form": np.sqrt(np.diag(np.linalg.inv(B_hat))),
    "Hessian": np.sqrt(np.diag(np.linalg.inv(A_hat))),
    "Sandwich": np.sqrt(
        np.diag(
            np.linalg.inv(A_hat) @ B_hat @ np.linalg.inv(A_hat)
        )
    ), 
    "Expected Information": np.sqrt(np.diag(np.linalg.inv(A_hat)))

}

pd.DataFrame(var_ests)

,Outer Product Form,Hessian,Sandwich,Expected Information
0,0.345240,0.303520,0.281102,0.303520
1,0.204950,0.226141,0.255106,0.226141
2,0.019246,0.018741,0.018834,0.018741
3,0.012983,0.009458,0.006949,0.009458
4,0.004443,0.003817,0.003306,0.003817


# Parts C) and D)

In [18]:
# from inspection
idx_cost = 4 
idx_ovtt = 2 

beta_cost = beta[idx_cost]
beta_ovtt = beta[idx_ovtt]

Vhat = beta[idx_cost] / beta[idx_ovtt]

var_cost  = np.linalg.inv(A_hat)[idx_cost, idx_cost]
var_ovtt  = np.linalg.inv(A_hat)[idx_ovtt, idx_ovtt]
cov_cost_ovtt = np.linalg.inv(A_hat)[idx_cost, idx_ovtt]

var_Vhat = (
    (var_cost / beta_ovtt**2)
    + (beta_cost**2 * var_ovtt / beta_ovtt**4)
    - (2 * beta_cost * cov_cost_ovtt / beta_ovtt**3)
)

se_Vhat = np.sqrt(var_Vhat)

# 95% CI
ci_lower = Vhat - 1.96 * se_Vhat
ci_upper = Vhat + 1.96 * se_Vhat

print(f"V_OVTT estimate = {Vhat:.4f}")
print(f"SE (Delta, expected info) = {se_Vhat:.4f}")
print(f"95% CI = ({ci_lower:.4f}, {ci_upper:.4f})")

V_OVTT estimate = 0.2723
SE (Delta, expected info) = 0.1206
95% CI = (0.0359, 0.5087)
